In [1]:
import findspark
findspark.init()

In [216]:
from pyspark.sql import SparkSession
from pyspark.sql import functions as F
from pyspark.sql import Window
from pyspark.sql.functions import udf

In [206]:
spark = SparkSession.builder.appName('Lab3').master('local[*]').getOrCreate()

In [207]:
data = spark.read.csv('data.csv', header=True, inferSchema=True)

In [208]:
data.printSchema()

root
 |-- _c0: integer (nullable = true)
 |-- date: string (nullable = true)
 |-- file_name: string (nullable = true)
 |-- time: double (nullable = true)
 |-- target_is_broken: boolean (nullable = true)
 |-- location: string (nullable = true)


In [209]:
data = data.drop('_c0').where(F.col('location') != '0').sort(['date', 'file_name', 'time'])

In [210]:
windowSpec = Window.partitionBy("date", "file_name").orderBy("time")

data = (data
    .withColumn('group_row', F.row_number().over(Window.partitionBy("date", "file_name").orderBy('time')))
    .withColumn('max_group', F.max('group_row').over(Window.partitionBy("date", "file_name")))
    .withColumn('is_window_last', F
        .when(F.col('max_group') == F.col('group_row'), 'Last')
        .otherwise('Not Last')
    )
 )

In [211]:
data = data.where(F.col('is_window_last') == 'Last')

In [212]:
data.show(n=10)

+----------+---------+----+----------------+--------------------+---------+---------+--------------+
|      date|file_name|time|target_is_broken|            location|group_row|max_group|is_window_last|
+----------+---------+----+----------------+--------------------+---------+---------+--------------+
|02-01-2024| SHOT0001| 4.5|           false|(1650.380859375, ...|       10|       10|          Last|
|02-01-2024| SHOT0002| 4.5|            true|(1973.966796875, ...|        9|        9|          Last|
|02-01-2024| SHOT0003| 5.0|           false|(2349.24169921875...|       10|       10|          Last|
|02-01-2024| SHOT0004| 4.5|            true|(1035.806640625, ...|        8|        8|          Last|
|02-01-2024| SHOT0005| 4.5|           false|(1653.04028320312...|        9|        9|          Last|
|02-01-2024| SHOT0006| 4.0|            true|(1364.68627929687...|        8|        8|          Last|
|02-01-2024| SHOT0007|13.0|           false|(1287.18823242187...|       21|       21|      

In [203]:
# Определяем оконную функцию
window_spec = Window.partitionBy('date').orderBy('file_name')

# Добавляем столбец с предыдущими значениями
df = data.withColumn("prev_value", F.lag("target_is_broken", 1).over(window_spec))

# Определяем, где значения идут подряд
df = df.withColumn("is_consecutive", F.when(F.col("target_is_broken") == F.col("prev_value"), 1).otherwise(0))

# Определяем группы подряд идущих значений
df = df.withColumn("group", F.sum("is_consecutive").over(Window.partitionBy('date').orderBy('file_name').rowsBetween(Window.unboundedPreceding, 2)))

# Считаем размер каждой группы
df = df.groupBy("target_is_broken", "group").count()

# Находим максимальный размер группы
max_consecutive_repeats = df.groupBy("target_is_broken").agg(F.max("count").alias("max_consecutive_repeats"))

# Показываем результат
max_consecutive_repeats.where(F.col('target_is_broken') == True).show()

+----------------+-----------------------+
|target_is_broken|max_consecutive_repeats|
+----------------+-----------------------+
|            true|                     10|
+----------------+-----------------------+


### Теперь с помощью pandas

In [220]:
import pandas as pd

In [221]:
data = pd.read_csv('data.csv').sort_values(by=['date', 'file_name', 'time'])

In [222]:
data.date.unique()

array(['02-01-2024', '02-07-2024', '02-18-2024', '02-21-2024',
       '03-01-2024', '03-07-2024', '03-14-2024', '03-24-2024',
       '03-28-2024'], dtype=object)

In [223]:
data_ = data[data.location != '0']

In [224]:
d = data_.groupby(by=['date', 'file_name']).tail(1).reset_index()

In [228]:
# Функция для нахождения максимального количества повторяющихся подряд значений
def max_consecutive_repeats(series):
    max_count = count = 1
    for i in range(1, len(series)):
        if series[i] == series[i - 1] == True:
            count += 1
        else:
            max_count = max(max_count, count)
            count = 1
    return max(max_count, count)


# Применяем функцию к нужному столбцу
for date in d.date.unique():
    max_repeats = max_consecutive_repeats(d[d.date == date].target_is_broken.reset_index(drop=True))

    print(f'Максимальное количество попаданий без промахов {max_repeats} на момент {date}')


Максимальное количество попаданий без промахов 8 на момент 02-01-2024
Максимальное количество попаданий без промахов 6 на момент 02-07-2024
Максимальное количество попаданий без промахов 9 на момент 02-18-2024
Максимальное количество попаданий без промахов 6 на момент 02-21-2024
Максимальное количество попаданий без промахов 4 на момент 03-01-2024
Максимальное количество попаданий без промахов 3 на момент 03-07-2024
Максимальное количество попаданий без промахов 1 на момент 03-14-2024
Максимальное количество попаданий без промахов 7 на момент 03-24-2024
Максимальное количество попаданий без промахов 10 на момент 03-28-2024
